In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 10:49:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 10:49:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/05 10:49:19 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


# Reduce : 여러개의 값을 하나로 축약
```
RDD.reduce( <func> )

```

In [3]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
from operator import add

In [5]:
sample_rdd.reduce(add)

15

# partition 이 나뉘어 져 있을 때

```
lambda x, y : (x*2) + y
```

1: x=1, y=2   >> (1*2) + 2 = 4
2: x=4, y=3   >> (4*2) + 3 = 11
3: x=11, y=4  >> (11*2) + 4 = 26
4:


In [6]:
##파티션 1개일 때 계산결과 26
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x,y: (x*2) + y)

26

In [10]:
#파티션 1개일 때
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [12]:
##파티션 2개일 때 계산결과 18
sample_rdd2 = spark.parallelize([1,2,3,4],2)
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

파티션1 
1: 1*2 + 2 = 4

파티션2
1: 3*2 + 4 = 10

최종결합:
x=4, y=10 : 4*2 + 10 = 18

In [13]:
sample_rdd2.reduce(lambda x,y: (x*2) + y)

18

In [ ]:
##파티션 3개일 때
p1 : 1
p2 : 2
p3 : 3,4 > 10
x=3, y=4 : 0+3*2 + 4 = 10

#p1, p2 결합
x=1, y=2 : 1*2+2 = 4

#4와 p3가 결합
x=4, y=10 : 4*2+10 = 18


In [14]:
sample_rdd3 = spark.parallelize([1,2,3,4],3)
sample_rdd3.glom().collect()

[[1], [2], [3, 4]]

In [15]:
sample_rdd3.reduce(lambda x,y: (x*2) + y)

18

### 리듀스 연산은 순서 의존적

각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합

### fold 연산의 활용

```
fold( zeroValue,  <func> )

```

zeroValue : 시작값, 항등원 *일때 1, +일 경우 0


In [16]:
rdd4 = spark.parallelize([2,3,4],4)
rdd4.reduce(lambda x,y : x*y)

24

In [17]:
rdd4.fold(1, lambda x,y: x*y)

24

In [20]:
#빈 RDD의 예외처리
rdd5 = spark.parallelize([])
try :
    rdd5.reduce(lambda a,b : a+b)
except ValueError as e:
    print(f"Reduce Error : {e}")

Reduce Error : Can not reduce() empty RDD


In [21]:
rdd5.fold(0, lambda a,b : a+b)

0

##  reduce(), fold() 비슷한 처리를 합니다.
1. reduce() 는 단순한 RDD 축약연산, 비어있지 않은 RDD 의 경우
2. fold() 는 일반적이고 오류가 없는 연산, 초기값을 적용해서 비어있는 RDD, 파티션의 경우에도 적용할 수 있음.

In [22]:
rdd6 = spark.parallelize([1,"a",,2])
rdd6.collect()

[1, 'a', 2]

In [36]:
#rdd6.reduce(lambda x,y: x+y)

24/12/05 11:47:59 ERROR Executor: Exception in task 0.0 in stage 27.0 (TID 38)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_9425/439935140.py", line 1, in <lambda>
TypeError: unsupported operand type(s) for +: 'int' and 'str'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$a

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 27.0 failed 1 times, most recent failure: Lost task 0.0 in stage 27.0 (TID 38) (ip-172-31-5-211.ap-northeast-3.compute.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_9425

In [40]:
from py4j.protocol import Py4JJavaError

In [43]:
try:
    rdd6.reduce(lambda x,y: x+y)
except Py4JJavaError as e:
    print(f"Reduce Error: ")

Reduce Error: 


24/12/05 11:53:06 ERROR Executor: Exception in task 0.0 in stage 32.0 (TID 43)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_9425/2801157106.py", line 2, in <lambda>
TypeError: unsupported operand type(s) for +: 'int' and 'str'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$

In [44]:
rdd6.fold("", lambda x,y: x + y)

24/12/05 11:55:26 ERROR Executor: Exception in task 0.0 in stage 33.0 (TID 44)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 1073, in func
    acc = op(acc, obj)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_9425/2512800836.py", line 1, in <lambda>
TypeError: can only concatenate str (not "int") to str

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunn

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 44) (ip-172-31-5-211.ap-northeast-3.compute.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 1073, in func
    acc = op(acc, obj)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_9425/2512800836.py"

In [ ]:
# a*b


In [35]:
1+2
print('hello')

hello


In [ ]:
# groupBy(), aggregate()

In [45]:
rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [47]:
result = rdd7.groupBy(lambda x: x%2 ).collect()

In [50]:
sorted([(x,sorted(y) ) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

In [ ]:
RDD.aggregate(zeroValue, func1_partion, func2_combine)

In [51]:
func1 = lambda x,y : (x[0] + y,  x[1] + 1)

In [52]:
func2 = lambda x,y : (x[0] + y[0], x[1] + y[1])

In [53]:
rdd8 = spark.parallelize( [1,2,3,4] , 2)

In [54]:
rdd8.aggregate( (0,0), func1, func2)

(10, 4)

In [55]:
rdd8.glom().collect()

[[1, 2], [3, 4]]

In [ ]:
p1
x=[1,2], zerovalue=[0,0]
x[0]=0, x[1]=0
x[0]+y = 0+1=1, x[1]+1 = 0+1=1  >> (1,1)
x[0]+y = 1+2=3, x[1]+1 = 1+1=2 >> (3,2)
p2
x=[3,4], zerovalue[0,0]
x[0]=0, x[1]=0
x[0]+y= 0+3=3, x[1]+1= 0+1=1 >> (3,1)
x[0]+y= 3+4=7, x[1]+1= 1+1=2 >> (7,2)
p1,p2 
x=(3,2), y=(7,2)
x[0]+y[0]=3+7=10, x[1]+y[1]=2+2=4 >> (10,4)

## key-value RDD (Paired RDD)

groupByKey(), reduceByKey()

그룹핑한 후에 특정 트랜스포메이션을 수행, 키가 필요함.

In [57]:
rdd9 = spark.parallelize(
    [
        ("짜장면", 15),
        ("짬뽕", 10),
        ("짜장면", 5),
        ("짬뽕", 20)
    ]
)

In [59]:
rdd9_g = rdd9.groupByKey()
rdd9_g

PythonRDD[74] at RDD at PythonRDD.scala:53

In [60]:
rdd9_g.mapValues(len).collect() #max, min

[('짜장면', 2), ('짬뽕', 2)]

In [61]:
rdd9_g.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [62]:
rdd9_g.mapValues(lambda x: sum(x) / len(x)).collect()

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [64]:
rdd9.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 30)]

In [ ]:
# reduceByKey() > groupByKey()

In [ ]:
#countByKey()
각 키별로 요소들의 갯수를 카운트 > Action 

In [65]:
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [ ]:
#key()  키만 가진 RDD > transformation

In [66]:
rdd9.keys()

PythonRDD[89] at RDD at PythonRDD.scala:53

In [67]:
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

# join() key 를 이용한 결합 - outer join, inner join

inner-join : 두개의 집합에서 함께 존재하는 요소(key)의 집합 (default)
outer-join : 한쪽에있고, 다른쪽에 없을 때 - left, right

In [73]:
join_rdd1 = spark.parallelize(
[
    ("a",10),
    ("b",20),
    ("c",30)

]
)

join_rdd2 = spark.parallelize(
[    ("a",20),
    ("b",20),
    ("b",10),
     ("d",5)
]
)

join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('a', (10, 20))]

In [74]:
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('c', (30, None)), ('a', (10, 20))]

In [75]:
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('b', (20, 10)), ('d', (None, 5)), ('a', (10, 20))]

In [76]:
spark.stop()